In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)



tensor([[ 0.2150,  0.2164,  0.2087,  0.1307,  0.0104,  0.0176,  0.0433, -0.1736,
          0.1823, -0.0165],
        [ 0.1812,  0.2159,  0.2304,  0.0711, -0.0840, -0.0536, -0.0152, -0.0862,
          0.0758, -0.0279]], grad_fn=<AddmmBackward0>)

In [6]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [7]:
net = MLP()
net(X)

tensor([[-0.0270, -0.2136, -0.1673, -0.3048,  0.0515,  0.1146, -0.0772, -0.1475,
          0.0077, -0.0511],
        [-0.0515, -0.0957, -0.0872, -0.1881, -0.0482,  0.0378, -0.1159, -0.0689,
         -0.0482, -0.0754]], grad_fn=<AddmmBackward0>)

In [10]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module, in enumerate(args):
            self._modules[str(idx)] = module
        
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [11]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.3458, -0.1079, -0.0297, -0.1413,  0.0678,  0.2027, -0.0125,  0.0027,
          0.1562,  0.0574],
        [ 0.0480, -0.1797,  0.0334, -0.1818,  0.1559,  0.1589, -0.0798,  0.1085,
          0.1641,  0.1126]], grad_fn=<AddmmBackward0>)

In [12]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)

        X = F.relu(F.relu(torch.mm(X, self.rand_weight) + 1))
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [13]:
net = FixedHiddenMLP()
net(X)

tensor(-0.0268, grad_fn=<SumBackward0>)

In [14]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1905, grad_fn=<SumBackward0>)